In [2]:
from client import Client
from server import Server
import os
import dgl
from time import time
import sys

# from ourparse import *
from scipy.sparse import lil_matrix, csr_matrix
from model import model
from rec_dataset import *
import torch.utils.data as dataloader
import copy
import random
from util import *

/opt/conda/envs/cu110/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using backend: pytorch


以mean归一化user-tag关系的pertubation的结果是不是一定程度有道理的。

加载数据

In [6]:
data_path = './data/100k/100k_mean.mat'
data = sio.loadmat(data_path)
# p_vs_f = data['PvsL']# (12499, 73)
p_vs_a = data['user_vs_movie']#(12499, 17431) (943, 1682)
# p_vs_t = data['PvsT']#(12499, 1903)
p_vs_c = data['user_vs_tag']#(12499, 14) (943, 19)
a_vs_t = data['movie_vs_tag']#(17431, 14) (1682, 19)

选取case做分析（用协同过滤的方法算一下两个user的相似度）

In [11]:
# user 0
true = [491, 302, 487, 550, 389, 291, 92, 289, 1045, 432, 431, 672, 10, 649, 947, 1006, 329, 423, 656, 0, 221, 528, 522, 380, 365, 512, 86, 834, 632, 521, 377, 363, 179, 665, 321, 438, 83, 751]
pertub = [  12,   22,   25 ,  29,   40,   77,   83 ,  86 ,  94 , 102 , 114 , 133,  164 , 170,
  200 , 209,  221 , 256 , 288,  289,  304,  329 , 333 , 380 , 423,  492 , 512 , 550,
  570 , 608 , 652,  659 , 695 , 719  ,779  ,807 , 862,  976, 1006, 1020, 1117 ,1169,
 1237, 1545,1594]
true_inters = np.zeros(1682)
pertub_inters = np.zeros(1682)

true_inters[true] = 1
pertub_inters[pertub] = 1

a_vs_t_np = np.array(a_vs_t.todense())
a_vs_t_np

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [19]:
true_tags = np.array(p_vs_c.todense())[0]

true_tags

array([0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [20]:
pertub_tags = np.dot(pertub_inters, a_vs_t_np)
# pertub_tags[pertub_tags > 0] = 1
pertub_tags

array([ 0., 10.,  5.,  0.,  1., 10.,  1.,  1., 20.,  0.,  0.,  2.,  0.,
        0., 12.,  7.,  9.,  2.,  0.])

In [21]:
from scipy.stats import pearsonr
pearsonr(true_tags, pertub_tags)

PearsonRResult(statistic=0.7717908764845521, pvalue=0.00010857587963080392)